In [3]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [4]:
### Maine Committee Search ###
#Set search for all Maine committees
state = 'ME'
office = 'P'
cycle='2020'


#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)
p=int(1)

querydict = {'api_key':config.api_key
            ,'state':state
            ,'office':office
            ,'cycle':cycle
            ,'per_page':'100'
            ,'page':p}
    
r = requests.get('https://api.open.fec.gov/v1/committees/',params=querydict).json()
    
#Loop through pages
while p <= r['pagination']['pages']:

    print('page: '+str(p)+','+str(r['pagination']['per_page'])+' records')

    querydict.update(page=p)
    #Pull results
    r = requests.get('https://api.open.fec.gov/v1/committees/',params=querydict).json()

    #Add results to dataframe
    df = json_normalize(r['results'])
    dfs.append(df)
    p=p+1

#Reset to page 1
p=int(1)
#Increment to next candidate
id=id+1

print('Job Complete')

page: 1,100 records
Job Complete


In [5]:
pres = df[df['committee_type_full']=='Presidential']
pres

,first_file_date,state,filing_frequency,affiliated_committee_name,last_file_date,organization_type,designation_full,committee_type,committee_type_full,name,designation,party_full,treasurer_name,party,candidate_ids,cycles,last_f1_date,organization_type_full,committee_id
7,2018-06-18,ME,Q,NONE,2018-06-18,None,Principal campaign committee,P,Presidential,"BRUCE JOHN KENNEWAY, INDEPENDENT PRESIDENT, NO...",P,WRITE-IN,"KENNEWAY, BRUCE JOHN MR",W,[P00007856],"[2018, 2020]",2018-06-18,None,C00681205
9,2018-08-22,ME,Q,NONE,2018-08-22,None,Principal campaign committee,P,Presidential,CALEB GAUL FOR PRESIDENT,P,DEMOCRATIC PARTY,"GAUL, CALEB",DEM,[P00008169],"[2018, 2020]",2018-08-22,None,C00685867
10,2019-08-16,ME,Q,NONE,2019-10-15,None,Principal campaign committee,P,Presidential,CAMPAIGN COMMITTEE TO ELECT EVRET J GREER,P,LIBERTARIAN PARTY,"GREER, EVRET JUSTIN",LIB,[P00013169],[2020],2019-08-16,None,C00715995
11,2019-07-09,ME,T,None,2019-10-11,None,Principal campaign committee,P,Presidential,CITIZENS FOR TOM KAWCZYNSKI,P,REPUBLICAN PARTY,THOMAS J KAWCZYNSKI,REP,[P00012708],[2020],2019-07-09,None,C00711606
24,2017-05-22,ME,T,NONE,2019-11-12,None,Principal campaign committee,P,Presidential,FRIENDS OF FRED WIAND,P,DEMOCRATIC PARTY,"WIAND, FRED",DEM,[P00005942],"[2018, 2020]",2017-05-22,None,C00641100


In [7]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
pres.to_csv(cwd+'/data/maine-presidential-candidates.csv')